# neural network train

In [1]:
#import libraries and packages
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
#translate from the matlab code
import xlrd
import pandas as pd

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from keras.models import Sequential
from keras.layers import Dense, Activation

#translate from the matlab code
import xlrd
import pandas as pd
import numpy as np

NumSKU=49
NumVar=13
ID=[1138263,1139362,1139363,1141061,1142731,1143640,1144140,
        1148001,1148010,1148081,1162466,1162467,1162557,1162558,
        1162559,1163152,1163153,1164313,1164961,1164962,1165757,
        1166153,1166984,1166998,1167021,1167087,1167847,1167918,
        1170236,1170372,1170739,1173299,1174241,1174242,1174243,
        1174244,1174275,1174293,1174299,1174313,1174314,1174315,
        1174339,1174340,1175687,1175833,1175835,1175950,1177151]

#train data
new_tv_info = pd.read_excel('C:/Users/wyd15/Downloads/Television.xlsx', sheet_name='new_tv_info') #processed by matlab
tv_sim = pd.read_excel('Television.xlsx', sheet_name='Similarity')
#tv_sim.head()

Using TensorFlow backend.


In [2]:
print(len(new_tv_info))
new_tv_info.head()

24697


,ID,Date,SalesQuantity,SalesQuantityLag1,SalesQuantityLag7,SalesQuantityLag14,Price,Discount,InventoryAvailability,WeekOfYear,...,Var75,Var76,Var77,Var78,Var79,Var80,Var81,Var82,Var83,Var84
0,1,2016-01-01,2,1,1,1,2626.27000,1.01,0.93,1,...,1779.66000,1.01,2329.659900,1.01,2533.055050,1.01,2541.530000,1.01,1753.387550,1.01
1,1,2016-01-02,6,2,1,1,2651.69398,1.01,0.93,1,...,1880.51008,1.01,2365.113317,1.01,2555.651667,1.01,2794.132285,1.01,1842.372600,1.01
2,1,2016-01-03,5,6,1,1,2774.57500,1.01,0.93,1,...,1851.69608,1.01,2355.083980,1.01,2725.285017,1.01,2835.449192,1.01,1789.243869,1.01
3,1,2016-01-04,6,5,1,1,2795.76000,1.01,0.93,2,...,1876.27505,1.01,2468.837450,1.01,2965.260000,1.01,2721.716275,1.01,1905.935100,1.01
4,1,2016-01-05,2,6,1,1,2795.76000,1.01,0.93,2,...,1876.27505,1.01,2468.837450,1.01,2626.273333,1.01,3473.720000,1.01,1905.925200,1.01


## Sequential Model

In [3]:
import tensorflow as tf
import numpy as np


def get_placeholder(input1_dim=18, input2_dim=61):
    return tf.placeholder(shape=(None, input1_dim), dtype=tf.float32), \
           tf.placeholder(shape=(None, input2_dim), dtype=tf.float32), \
           tf.placeholder(shape=(None, ), dtype=tf.float32)


def get_output(input1, input2):
     
    input1_dim = input1.get_shape()[1]
    input2_dim = input2.get_shape()[1]
    W1_1 = tf.get_variable('W1_1', shape=(input1_dim, input1_dim+input2_dim))
    W1_2 = tf.get_variable('W1_2', shape=(input2_dim, input1_dim+input2_dim))
    b1 = tf.get_variable('b1', shape=(1, input1_dim+input2_dim))
    
    o1_1 = tf.matmul(input1, W1_1)
    o1_2 = tf.matmul(input2, W1_2)
    o1 = o1_1 + o1_2 + b1
    o1 = tf.nn.sigmoid(o1)

    W2_1 = tf.get_variable('W2_1', shape=(input2_dim, 1))
    W2_2 = tf.get_variable('W2_2', shape=(input1_dim+input2_dim, 1))
    b2 = tf.get_variable('b2', shape=(1, 1))
    o2_1 = tf.matmul(input2, W2_1)
    o2_2 = tf.matmul(o1, W2_2)
    o2 = o2_1 + o2_2 + b2
    o2 = tf.reshape(o2, (-1, ))
    
    saver = tf.train.Saver([W1_1, W1_2, b1, W2_1, W2_2, b2])
    Vars=[W1_1, W1_2, b1, W2_1, W2_2, b2]
    
    return o2, saver, Vars


def get_loss(labels, predictions):
    return tf.losses.mean_squared_error(labels, predictions)


def main():
    tf.reset_default_graph()
    
    batch_size = 50
    learning_rate = 0.11
    tf.reset_default_graph()
    x1, x2, y = get_placeholder(18, 61)
    output, saver, Vars = get_output(x1, x2)

    loss = get_loss(y, output)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    #train-test_splie
    msk = np.random.rand(len(new_tv_info)) < 0.85
    tv_train = new_tv_info[msk]
    tv_test = new_tv_info[~msk]
    
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for step in range(493):
            x1_train = tv_train.iloc[step:step+batch_size, 66:84].values 
            x2_train = np.concatenate((tv_train.iloc[step:step+batch_size, 3:11].values, tv_train.iloc[step:step+batch_size, 14:67].values), 1) 
            y_train = tv_train.iloc[step:step+batch_size, 2].values 
            
            x1_test = tv_test.iloc[step:step+batch_size, 66:84].values 
            x2_test = np.concatenate((tv_test.iloc[step:step+batch_size, 3:11].values, tv_test.iloc[step:step+batch_size, 14:67].values), 1) 
            y_test = tv_test.iloc[step:step+batch_size, 2].values 
            
            _train_loss, _ = sess.run([loss, train_op],
                                   feed_dict={x1: x1_train,
                                              x2: x2_train,
                                              y: y_train})
            
            _test_loss= sess.run([loss],
                                   feed_dict={x1: x1_test,
                                              x2: x2_test,
                                              y: y_test})
            
            #results, _ = sess.run([output]) #print parameters, W1_1 etc..
            
            if step % 100==0:
                saver.save(sess, 'C:/Users/wyd15/Desktop/tv_model/tv_modelslack.ckpt', global_step=step)
                print("iter: %d, train_loss: %f, test_loss: %f"%(step, _train_loss,  _test_loss[0]))
                #print('output:', )


if __name__ == '__main__':
    main()

iter: 0, train_loss: 22379.767578, test_loss: 145694.078125
iter: 100, train_loss: 261.362335, test_loss: 67290.007812
iter: 200, train_loss: 255.445541, test_loss: 4472.424805
iter: 300, train_loss: 44.787533, test_loss: 516.710327
iter: 400, train_loss: 1.891456, test_loss: 23.775335


# original tv demand neural network

In [4]:
# tensorflow nn
#unfolded version
#reference code: https://github.com/yl3829/Spring2018-Project5-grp5/blob/master/lib/model_tb.py  

# D = tf.placeholder(dytpe=np.floate32)
# w1_1 = tf.Variable((18,79),name='W1_1')
# w1_2 = tf.Variable((61,79),name='W1_2')
# b1 = tf.Variable((79,),name='b1')
# param = [w1_1,w1_2,b1]
tf.reset_default_graph()
x1=tf.placeholder(shape=(None, 18), dtype=tf.float32)
x2=tf.placeholder(shape=(None, 61), dtype=tf.float32)
y =tf.placeholder(shape=(None, ), dtype=tf.float32)

#
W1_1 = tf.get_variable('W1_1', shape=(18, 79))
W1_2 = tf.get_variable('W1_2', shape=(61, 79))
b1 = tf.get_variable('b1', shape=(1, 79))
    
o1_1 = tf.matmul(x1, W1_1)
o1_2 = tf.matmul(x2, W1_2)
o1 = o1_1 + o1_2 + b1
o1 = tf.nn.sigmoid(o1)

W2_1 = tf.get_variable('W2_1', shape=(61, 1))
W2_2 = tf.get_variable('W2_2', shape=(79, 1))
b2 = tf.get_variable('b2', shape=(1, 1))

o2_1 = tf.matmul(x2, W2_1)
o2_2 = tf.matmul(o1, W2_2)
o2 = o2_1 + o2_2 + b2
o2 = tf.reshape(o2, (-1, ))

saver = tf.train.Saver([W1_1, W1_2, b1, W2_1, W2_2, b2])
#

loss = tf.losses.mean_squared_error(y, o2)
optimizer = tf.train.AdamOptimizer(learning_rate=0.11)
train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
# layer1_out  = tf.nn.sigmoid(tf.matmul(input1,w1_1) + tf.matmul(input2,w1_2) + b1) # tanh would be better
# # example to minimize layer1 out put
# losses = layer2_out - D
# loss = tf.reduce_mean(losses)
# trainer = tf.train.AdamOptimizer()
# gradients = trainer.compute_gradients(self.loss)
# optimizer = trainer.apply_gradients(gradients)

#train-test_splie
msk = np.random.rand(len(new_tv_info)) < 0.85
tv_train = new_tv_info[msk]
tv_test = new_tv_info[~msk]
batch_size=50

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(493):
        x1_train = tv_train.iloc[step:step+batch_size, 66:84].values 
        x2_train = np.concatenate((tv_train.iloc[step:step+batch_size, 3:11].values, tv_train.iloc[step:step+batch_size, 14:67].values), 1) 
        y_train = tv_train.iloc[step:step+batch_size, 2].values 
            
        x1_test = tv_test.iloc[step:step+batch_size, 66:84].values 
        x2_test = np.concatenate((tv_test.iloc[step:step+batch_size, 3:11].values, tv_test.iloc[step:step+batch_size, 14:67].values), 1) 
        y_test = tv_test.iloc[step:step+batch_size, 2].values 
            
        _train_loss, _ = sess.run([loss, train_op],feed_dict={x1: x1_train,
                                              x2: x2_train,
                                              y: y_train})
            
        _test_loss= sess.run([loss],feed_dict={x1: x1_test,
                                              x2: x2_test,
                                              y: y_test})
        # predict
        D_predict = sess.run([o2], feed_dict={x1:x1_test,x2:x2_test})
        para_pred = sess.run([W1_1, W1_2, b1, W2_1, W2_2, b2], feed_dict={x1:x1_test,x2:x2_test})
        #results, _ = sess.run([output]) #print parameters, W1_1 etc..
            
        if step % 100==0:
            saver.save(sess, 'C:/Users/wyd15/Desktop/tv_model/tv_modelslack.ckpt', global_step=step)
            print("iter:%d, train_loss: %f, test_loss: %f"%(step, _train_loss,  _test_loss[0]))
            print('test prediction parameters results:', D_predict)
            #print('predicted parameters:', para_pred)
    print('last test prediction parameters results:', step, _train_loss,  _test_loss[0], D_predict)
    paras=para_pred
                #print('output:', )

iter:0, train_loss: 484165.875000, test_loss: 13285.414062
test prediction parameters results: [array([ 142.6806488 ,  144.60281372,  144.73997498,  134.95324707,
        138.21200562,  134.62776184,  140.89408875,  134.02508545,
        134.77386475,  140.45220947,  138.36047363,  146.27722168,
        130.59552002,  130.54397583,  151.14276123,  131.98068237,
        131.17425537,  134.09301758,  135.7194519 ,  115.26747894,
        137.05897522,  123.21807098,  121.61158752,  121.65653992,
        120.83131409,  112.87034607,  124.26582336,  117.22241974,
        118.47533417,  122.93070221,  122.1912384 ,  120.49742126,
        114.342453  ,  113.62680054,  121.43595886,  103.77321625,
        114.78121185,  114.32935333,  108.43563843,  108.81217194,
        109.18531799,   97.17289734,  100.35605621,   96.27137756,
         97.4725647 ,   84.61821747,   84.87034607,   85.0085144 ,
         84.57138062,   84.58546448], dtype=float32)]
iter:100, train_loss: 79.309921, test_loss: 17

# printing tv nn trained parameters

In [5]:
import numpy as np
np.set_printoptions(threshold=1500)
import pickle
para_names=['W1_1', 'W1_2', 'b1', 'W2_1', 'W2_2', 'b2']
for i in range(len(para_names)):
    print(' ===== printing paramter: '+ para_names[i] +' ===== ')
    print(para_pred[i])

 ===== printing paramter: W1_1 ===== 
[[ -5.18386364e-02   8.38602930e-02  -1.80666959e+00   1.10418415e+00
   -2.07713336e-01  -2.21424723e+00  -2.44942665e-01  -4.10063267e-02
   -9.75947380e-02   8.58362764e-02  -1.22434035e-01   1.89107850e-01
   -9.15461332e-02   2.25905493e-01   6.66529477e-01   1.60307109e+00
    1.73369154e-01  -1.64197087e-02  -8.60521793e-02  -6.75001621e-01
    9.11085159e-02  -1.83954120e+00   1.59876630e-01   4.77414876e-02
    5.18515527e-01  -2.44292289e-01   1.11651465e-01   4.39683169e-01
   -1.10453701e+00  -2.40049273e-01  -1.41516089e-01  -1.58604667e-01
    7.61337578e-03  -2.41686210e-01   2.01219583e+00   1.25279856e+00
    1.37263775e+00  -1.51915237e-01  -1.34219289e-01   1.71839669e-01
   -1.11546290e+00   2.30179280e-02   6.92257136e-02  -6.31567359e-01
   -1.05009302e-01   1.65775701e-01   1.88918263e-02  -5.79876363e-01
   -5.75691760e-02   1.97266951e-01   7.24060088e-02   1.63395837e-01
    1.55686259e+00  -2.38423645e-01  -1.35819423e+00

# plug in parameters from tv nn and train optimization nn

simulation note:
- ignored constant vector for the optimization, i.e. set as zero vector
- price_focal trained here has length 61 as in tv_nn

In [15]:
x2_train[:, 3]

array([ 2539.83,  2539.83,  2539.83,  2539.83,  2539.83,  2539.83,
        2539.83,  2539.83,  2539.83,  2539.83,  2539.83,  2539.83,
        2539.83,  2539.83,  2539.83,  2539.83,  2539.83,  2539.83,
         555.19,   555.19,   555.19,   555.19,   555.19,   555.19,
         555.19,   555.19,   555.19,   555.19,   555.19,   555.19,
         555.19,   555.19,   555.19,   555.19,   555.19,   555.19,
         555.19,   555.19,   555.19,   555.19,   555.19,   555.19,
         555.19,   555.19,   555.19,   555.19,   555.19,   555.19,
         555.19,   555.19])

In [ ]:
import time 
#optimize the loss funciton by adding negative sign
start_time= time.time()
tf.reset_default_graph()
#p_f = tf.get_variable('p_f', shape=(1, 61)) 

p_f = tf.get_variable('p_f', shape=(1, 1)) #corresponding index 3 in 

W1_1 = tf.constant(paras[0], dtype=tf.float32, name='W1_1' )
W1_2 = tf.constant(paras[1], dtype=tf.float32, name='W1_2' )
b1 = tf.constant(paras[2], dtype=tf.float32, name='b1' )
W2_1 = tf.constant(paras[3], dtype=tf.float32, name='W2_1' )
W2_2 = tf.constant(paras[4], dtype=tf.float32, name='W2_2' )
b2 = tf.constant(paras[5], dtype=tf.float32, name='b2' )
p_sub =  tf.placeholder(shape=(None, 18), dtype=tf.float32)
D = tf.matmul(tf.nn.sigmoid(tf.matmul(p_f, W1_2) + tf.matmul(p_sub, W1_1)+b1), W2_2)+tf.matmul(p_f, W2_1)+b2

p_loss = tf.reduce_mean(-p_f*D) #self defined loss

###parameters
batch_size = 50
learning_rate = 0.5
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
gradients = trainer.compute_gradients(p_loss)
optimizer = trainer.apply_gradients(gradients)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(493):
        #parameters 
        x1_train = tv_train.iloc[step:step+batch_size, 66:84].values 
        x1_test = tv_test.iloc[step:step+batch_size, 66:84].values

        
        loss_tr, _ = sess.run([p_loss, optimizer], feed_dict={p_sub: x1_train})
        _test_loss = sess.run([p_loss], feed_dict={p_sub: x1_test})
        
        # predict
        D_pred_tr = sess.run([D], feed_dict={p_sub:x1_train})
        D_pred_te = sess.run([D], feed_dict={p_sub:x1_test})
        pf_pred_tr = sess.run([p_f], feed_dict={p_sub:x1_train})
        pf_pred_te = sess.run([p_f], feed_dict={p_sub:x1_test})
        
        if step % 100 == 0:
            print("iter:%d, train_loss: %f, test_loss: %f"%(step, loss_tr,  _test_loss[0]))
            print("simulated train_revenue: %f, simulated test_revenue: %f"%(-loss_tr,  -_test_loss[0]))
            print('train D prediction parameters results:', D_pred_tr[0][0])
            print('test D prediction parameters results:', D_pred_te[0][0])
            print('train p_f prediction parameters results:', pf_pred_tr[0][0])
            print('test p_f prediction parameters results:', pf_pred_te[0][0])
            
end_time=time.time()

print('trained total elasped time:', end_time-start_time)

In [76]:
import time 
#optimize the loss funciton by adding negative sign
start_time= time.time()
tf.reset_default_graph()
p_f = tf.get_variable('p_f', shape=(1, 61)) 


W1_1 = tf.constant(paras[0], dtype=tf.float32, name='W1_1' )
W1_2 = tf.constant(paras[1], dtype=tf.float32, name='W1_2' )
b1 = tf.constant(paras[2], dtype=tf.float32, name='b1' )
W2_1 = tf.constant(paras[3], dtype=tf.float32, name='W2_1' )
W2_2 = tf.constant(paras[4], dtype=tf.float32, name='W2_2' )
b2 = tf.constant(paras[5], dtype=tf.float32, name='b2' )
p_sub =  tf.placeholder(shape=(None, 18), dtype=tf.float32)
D = tf.matmul(tf.nn.sigmoid(tf.matmul(p_f, W1_2) + tf.matmul(p_sub, W1_1)+b1), W2_2)+tf.matmul(p_f, W2_1)+b2

p_loss = tf.reduce_mean(-p_f*D) #self defined loss

###parameters
batch_size = 50
learning_rate = 0.5
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
gradients = trainer.compute_gradients(p_loss)
optimizer = trainer.apply_gradients(gradients)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(493):
        #parameters 
        x1_train = tv_train.iloc[step:step+batch_size, 66:84].values 
        x1_test = tv_test.iloc[step:step+batch_size, 66:84].values

        
        loss_tr, _ = sess.run([p_loss, optimizer], feed_dict={p_sub: x1_train})
        _test_loss = sess.run([p_loss], feed_dict={p_sub: x1_test})
        
        # predict
        D_pred_tr = sess.run([D], feed_dict={p_sub:x1_train})
        D_pred_te = sess.run([D], feed_dict={p_sub:x1_test})
        pf_pred_tr = sess.run([p_f], feed_dict={p_sub:x1_train})
        pf_pred_te = sess.run([p_f], feed_dict={p_sub:x1_test})
        
        if step % 100 == 0:
            print("iter:%d, train_loss: %f, test_loss: %f"%(step, loss_tr,  _test_loss[0]))
            print("simulated train_revenue: %f, simulated test_revenue: %f"%(-loss_tr,  -_test_loss[0]))
            print('train D prediction parameters results:', D_pred_tr[0][0])
            print('test D prediction parameters results:', D_pred_te[0][0])
            print('train p_f prediction parameters results:', pf_pred_tr[0][0])
            print('test p_f prediction parameters results:', pf_pred_te[0][0])
            
end_time=time.time()

print('trained total elasped time:', end_time-start_time)

iter:0, train_loss: 0.015254, test_loss: -1.931153
simulated train_revenue: -0.015254, simulated test_revenue: 1.931153
train D prediction parameters results: [ 4.08376312]
test D prediction parameters results: [ 4.08376312]
train p_f prediction parameters results: [ 0.27519163  0.71603501  0.40172035  0.2465378   0.65532756  0.72448307
  0.26340601  0.68892837  0.73887032  0.30339321  0.74439216  0.25574148
  0.58710772  0.22948661  0.37563109  0.48797232  0.25176153  0.32835129
  0.78147292  0.3213383   0.33693913  0.5167436   0.58838892  0.80850506
  0.44930342  0.28909355  0.59339476  0.48936957  0.23207715  0.70006233
  0.23459548  0.38393834  0.7051819   0.5100767   0.45062947  0.58456802
  0.54376048  0.52921164  0.31723985  0.60836881  0.74959129  0.61383998
  0.48322216  0.39377356  0.70933682  0.28857201  0.48687404  0.63943958
  0.38513982  0.4271408   0.6301102   0.28774959  0.75927979  0.2149156
  0.71745455  0.64149898  0.55777717  0.26488853  0.53295612  0.6465013
  0.34

trained total elasped time: 2.4205403327941895
